In [1]:
# -*- coding: utf-8 -*
import argparse
import torch
import torch.nn as nn
from flyai.dataset import Dataset
from torch.optim import Adam

from model import Model
from net import Net
from path import MODEL_PATH

# 超参
parser = argparse.ArgumentParser()
parser.add_argument("-e", "--EPOCHS", default=10, type=int, help="train epochs")
parser.add_argument("-b", "--BATCH", default=16, type=int, help="batch size")
args = parser.parse_args([])

clip = 5

# 判断gpu是否可用
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device = torch.device(device)


def eval(model, x_test, y_test):
    network.eval()
    total_acc = 0.0
    data_len = len(x_test[0])
    x1, x2 = x_test
    x1 = torch.from_numpy(x1)
    x2 = torch.from_numpy(x2)
    x1 = x1.float().to(device)
    x2 = x2.float().to(device)
    y_test = torch.from_numpy(y_test)
    y_test = y_test.to(device)
    batch_eval = model.batch_iter(x1, x2, y_test)

    for x_batch1, x_batch2, y_batch in batch_eval:
        outputs = network(x_batch1, x_batch2)
        _, prediction = torch.max(outputs.data, 1)
        correct = (prediction == y_batch).sum().item()
        total_acc += correct
    return total_acc / data_len

In [5]:
# 数据获取辅助类
data = Dataset()
network = Net().to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(network.parameters(), lr=0.001, weight_decay=1e-4)  # 定义优化器，选用AdamOptimizer

model = Model(data)
iteration = 0

In [ ]:
!which python

In [15]:
best_accuracy = 0
# 得到训练和测试的数据
for epoch in range(args.EPOCHS):
    network.train()

    # 得到训练和测试的数据
    x_train, y_train, x_test, y_test = data.next_batch(args.BATCH)  # 读取数据

    batch_len = y_train.shape[0]
    x1, x2 = x_train
    x1 = torch.from_numpy(x1)
    x2 = torch.from_numpy(x2)
    x1 = x1.float().to(device)
    x2 = x2.float().to(device)
    y_train = torch.from_numpy(y_train)
    y_train = y_train.to(device)

    outputs = network(x1, x2)
    _, prediction = torch.max(outputs.data, 1)

    optimizer.zero_grad()
    outputs = outputs.float()
    # calculate the loss according to labels
    loss = loss_fn(outputs, y_train)

    # backward transmit loss
    loss.backward()

    # adjust parameters using Adam
    optimizer.step()

    # 若测试准确率高于当前最高准确率，则保存模型
    train_accuracy = eval(model, x_test, y_test)
    print(train_accuracy)
    if train_accuracy > best_accuracy:
        best_accuracy = train_accuracy
        model.save_model(network, MODEL_PATH, overwrite=True)
        print("step %d, best accuracy %g" % (epoch, best_accuracy))

    print(str(epoch) + "/" + str(args.EPOCHS))

0.625
step 0, best accuracy 0.625
0/10
0.71875
step 1, best accuracy 0.71875
1/10
0.71875
2/10
0.46875
3/10
0.625
4/10
0.71875
5/10
0.71875
6/10
0.46875
7/10
0.625
8/10
0.71875
9/10


In [17]:
import sys

import codecs
import json
import numpy as np
import os
import re
from flyai.processor.base import Base

from path import DATA_PATH  # 导入输入数据的地址

#sys.stdout = codecs.getwriter("utf-8")(sys.stdout.detach())

In [34]:
a = data.db.source.data

In [19]:
 embedding_path = os.path.join(DATA_PATH, 'embedding.json')
with open(embedding_path, encoding='utf-8') as f:
    vocab = json.loads(f.read())
max_sts_len = 10
embedding_len = 100

In [29]:
v=vocab.get('path')

In [31]:
v[:5]

[0.20893999934196472,
 0.22754999995231628,
 0.17449000477790833,
 0.34053999185562134,
 0.9351699948310852]

In [32]:
def words2vec(words,vocab,embedding_len,max_sts_len):
    words = str(words)
    words = re.sub("[\s+\.\!\/_,$%^*()+-?\"\']+|[+——！，。；？、~@#￥%……&*（）]+", " ", words)
    words = words.strip().split(' ')
    
    vecs = []
    for word in words:
        embedding_vector = vocab.get(word)
        if embedding_vector is not None and len(embedding_vector) == embedding_len:
                vecs.append(embedding_vector)
    if len(vecs) >= max_sts_len:
        vecs = vecs[:max_sts_len]
    else:
        for i in range(len(vecs), max_sts_len):
            vecs.append([0 for j in range(embedding_len)])
    vecs = np.stack(vecs)
    return vecs

In [40]:
b=words2vec(a.iloc[0][0],vocab,100,10)

In [41]:
b.shape

(10, 100)

In [44]:
[1,2,]

NameError: name 'NaN' is not defined

In [46]:
from math import isnan

In [ ]:
import torch
import torch.nn as nn

num_dims = 100


class LSTMNet(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.LSTM_stack = nn.LSTM(num_dims, 64, num_layers=2, batch_first=True)
        for name, param in self.LSTM_stack.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)

        self.relu1 = nn.ReLU(True)
        self.fc1 = nn.Linear(10 * 64 * 2, 128)  ##  (max sentence length * hidden layer, 512)
        self.relu2 = nn.ReLU(True)
        self.dp = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 2)
        self.bn = nn.BatchNorm1d()

    def forward(self, x1, x2):
        x1, _ = self.LSTM_stack(x1.float())  # (batch, sentence_len, hidden_units)
        x2, _ = self.LSTM_stack(x2.float())
        x = torch.cat((x1,x2),dim=1)
        #x = self.dp(x)
        #x = self.bn(x)
        

        # use every word in the sentence
        #x = x.contiguous().view(-1, x.size(1) * x.size(2))
        x = self.relu1(x)
        x = self.fc1(x.float())
        x = self.relu2(x)
        x = self.dp(x)
        x = self.fc2(x)
        x = x / torch.norm(x)
        return x
        

In [ ]:
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)